In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [3]:

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn."
        }
    ]
)

print(completion.choices[0].message.content)

Under a silvery moon, a gentle unicorn named Luna tiptoed through a field of glowing flowers, leaving sparkling dreams in every child’s heart as they drifted off to sleep.


In [32]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [33]:
get_weather(28.65195, 77.23149)

30.3

![alt text](function-calling-diagram-steps.png)

In [14]:
import requests
import json

def get_current_weather(location):
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places_prefix"

    querystring = {"text":location,"language":"en"}

    headers = {
        "x-rapidapi-key": "01bd5caaaemsheb6f5da0dd2b2efp155ce9jsnd519fdbf076e",
        "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())

In [15]:
from openai import OpenAI
client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Retrieves current weather for the given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                },
                "units": {
                    "type": "string",
                    "enum": [
                        "celsius",
                        "fahrenheit"
                    ],
                    "description": "Units the temperature will be returned in."
                }
            },
            "required": [
                "location",
                "units"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [16]:
messages = [{"role": "user", "content": "What's the weather like in Delhi today?"}]

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [17]:
print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-BbJf8AJHvCSJ6PmZ14NGvQhImdKYh",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_B2fWyOXT6KDZlpxZi2PYeZSz",
            "function": {
              "arguments": "{\"location\":\"Delhi, India\",\"units\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1748233418,
  "model": "gpt-4.1-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_b3f1157249",
  "usage": {
    "completion_tokens": 22,
    "prompt_tokens": 84,
    "total_tokens": 106,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tok

In [21]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

# result = get_weather(arguments["location"])
print(args['location'])

result = get_current_weather(args['location'])

Delhi, India
[{'name': 'Delhi', 'place_id': 'delhi', 'adm_area1': 'National Capital Territory of Delhi', 'adm_area2': None, 'country': 'India', 'lat': '28.65195N', 'lon': '77.23149E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Najafgarh', 'place_id': 'najafgarh', 'adm_area1': 'National Capital Territory of Delhi', 'adm_area2': 'South West', 'country': 'India', 'lat': '28.60922N', 'lon': '76.97982E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'National Capital Territory of Delhi', 'place_id': 'national-capital-territory-of-delhi', 'adm_area1': 'National Capital Territory of Delhi', 'adm_area2': None, 'country': 'India', 'lat': '28.6667N', 'lon': '77.1E', 'timezone': 'Asia/Kolkata', 'type': 'administrative_area'}, {'name': 'North West Delhi', 'place_id': 'north-west-delhi', 'adm_area1': 'National Capital Territory of Delhi', 'adm_area2': 'North West Delhi', 'country': 'India', 'lat': '28.70113N', 'lon': '77.10154E', 'timezone': 'Asia/Kolkata', 'type': '

In [22]:
print(result)

None


In [ ]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
                
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [24]:
completion_2.choices[0].message.content

"I'm unable to retrieve the current weather for Delhi right now. Would you like a general weather overview for Delhi in June, or would you like to try again later for live weather information?"